In [1]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# # from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.chrome.options import Options
# import re
# import sys
# import time
# import numpy as np
# import pandas as pd
# import os
# import psycopg2
# from tqdm import tqdm


# def main():

#     docs = np.arange(10000,20000,1)
#     limit = 10

#     def html(url):
#         try:
#             driver.get(url)
#             html = driver.page_source
#             if type(html) == str:
#                 return(html)
#             else:
#                 return('ERROR')
#         except:
#             return('ERROR')

#     def headlessDriver():
#         options = Options()
#         options.add_argument("--headless")
#         options.add_argument('log-level=1')     
#         driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")
#         return(driver)


#     def buildEntry(index_value,url):
#         t_start = time.time()    
#         index = index_value
#         pageSource = html(url)
#         pageText = str()
#         if pageSource == 'ERROR':
#             pageText = 'ERROR'
#         else:
#             try:
#                 soup=BeautifulSoup(pageSource, 'lxml')
#                 pageText = soup.find(class_='content').text
#                 if pageText == '':
#                     pageText = None
#             except:
#                 pageText = 'ERROR'

#         entry = (pageSource,
#                  pageText,
#                  index)

#         t_end = time.time()
#         duration = t_end - t_start
#         if duration >= 10:
#             time.sleep(np.random.randint(5,15))          
#         return(entry)


#     def fullHarvest(doclist=docs, limit=limit):
#         t = time.time()

#         doclist = tuple(doclist)    
#         limit = 10

#         conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                             database="postgres",
#                             user="tokyo3",
#                             password="evangelion")

#         cur = conn.cursor()

#         command = """
#         SELECT doc_id,doc_url
#         FROM documents
#         WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_skip != 1 and doc_id IN {}
#         LIMIT {}
#         """.format(doclist,limit)

#         cur.execute(command)
#         existing = cur.fetchall()
#         cur.close()
#         conn.close()

#         submission = [buildEntry(int(x[0]),str(x[1])) for x in existing]    
#         skiplist = []
#         newSubmission = []

#         for i in submission:
#             if i[1] == 'ERROR':
#                 skiplist.append(i[2])
#             else:
#                 newSubmission.append(i)

#         sub_len = len(newSubmission)


#         conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                             database="postgres",
#                             user="tokyo3",
#                             password="evangelion")

#         cur = conn.cursor()

#         if len(skiplist) > 1:
#             sql = """
#             UPDATE documents
#             SET doc_skip = 1
#             WHERE doc_id IN {};
#             """.format(tuple(skiplist))
#             cur.execute(sql)
#             conn.commit()
#         if len(skiplist) == 1:
#             sql = """
#             UPDATE documents
#             SET doc_skip = 1
#             WHERE doc_id = {};
#             """.format(skiplist[0])
#             cur.execute(sql)
#             conn.commit()


#         sql = """
#             UPDATE documents
#             SET doc_html = %s,doc_content = %s
#             WHERE doc_id = %s;
#             """

#         cur.executemany(sql,newSubmission)
#         conn.commit()


#         command = """
#         SELECT COUNT(doc_id)
#         FROM documents
#         WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_skip != 1 and doc_id IN {}
#         """.format(doclist)
#         cur.execute(command)
#         count = cur.fetchall()
#         cur.close()
#         conn.close()

#         nt = time.time()
#         dt = nt - t   

#         return(len(skiplist),count,dt,sub_len,existing)


#     counter = 0
#     driver = headlessDriver()

#     for i in tqdm(range(1000)):    
#         s,r,t,l,e = fullHarvest(doclist=docs, limit=limit)
#         print("{} : {:.4f}s : {} skips : {} remaining : {} counter".format(i,t,s,r,counter))
#         if l == 0:
#             print(e)
#             counter += 1
#             if counter > 10:
#                 break
#             else:
#                 driver.quit()
#                 time.sleep(np.random.randint(30,45))
#                 driver = headlessDriver()
                
# if __name__ == '__main__':
#     main()



In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import re
import sys
import time
import numpy as np
import pandas as pd
import os
import psycopg2
from tqdm import tqdm


In [3]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT doc_id,doc_url
FROM documents

"""

cur.execute(command)
existing = cur.fetchall()
cur.close()
conn.close()


# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                     database="postgres",
#                     user="tokyo3",
#                     password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT doc_content, COUNT(doc_id)
# FROM documents
# WHERE doc_html IS NOT NULL
# GROUP BY doc_content
# HAVING COUNT(doc_id) > 1
# """

# cur.execute(command)
# existing = cur.fetchall()
# cur.close()
# conn.close()

In [4]:
len(existing)

125745

In [10]:
existing[:10]

[(8230,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676860/xgxw_676866/t156528.shtml'),
 (82296,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677848/1206x2_677868/t811819.shtml'),
 (82303,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677848/1206x2_677868/t758120.shtml'),
 (82309,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677848/1206x2_677868/t725971.shtml'),
 (82313,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677848/1206x2_677868/t724028.shtml'),
 (82315,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677848/1206x2_677868/t722354.shtml'),
 (8232,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676860/xgxw_676866/t61431.shtml'),
 (8233,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676860/xgxw_676866/t40464.shtml'),
 (82323,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1

In [11]:
existingList = [x[1] for x in existing]

In [34]:
gatheredList = pd.read_json('documents.json')
gatheredList.head()

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
10,博茨瓦纳,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_博茨瓦纳_驻外报道_,驻博茨瓦纳大使蒋正云向博茨瓦纳总统递交国书(2005-01-21),2005-01-21
100,美洲玻利瓦尔联盟,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_美洲玻利瓦尔联盟_相关新闻_,加拿大总理哈珀会见杨洁篪(2009-06-24),2009-06-24
1000,布隆迪,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_非洲_布隆迪_驻外报道_,驻布隆迪大使曾宪柒拜会布新任对外关系部长(2009-03-06),2009-03-06


In [31]:
newDocs = list(set(gatheredList['url']) - set(gatheredList['url']).intersection(set(existingList)))
newDF = gatheredList.copy()
setDocs = set(newDocs)
len(setDocs)

6435

In [36]:
newIndex = []
for i in newDF.index:
    if newDF.loc[i,'url'] in setDocs:
        newIndex.append(i)

In [38]:
uploadDF = newDF.loc[newIndex,].copy()

In [48]:
uploadDF.head()

,country,lang,page,url,nav,title,date
0,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
100025,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典使馆举办中瑞青少年羽毛球交流开放日活动(2019-01-28),2019-01-28
100148,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典大使桂从友在2019年瑞典中国学生学者春节联欢会上的致辞(2019-01-21),2019-01-21
100222,乌兹别克斯坦,zh,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_乌兹别克斯坦_外交掠影_,费尔干纳——名符其实的乌兹别克斯坦明珠(2018-11-08),2018-11-08


In [49]:
uploadDF['date'].describe()

count           6435
unique          1897
top       2019-01-23
freq              69
Name: date, dtype: object

In [97]:
# uploadDF = uploadDF.drop(columns='more')

In [98]:
uploadDF[uploadDF['date']!='9999-99-99'].shape

(6432, 7)

In [99]:
uploadDF.shape

(6435, 7)

In [80]:
uploadDF.head()

,country,lang,page,url,nav,title,date
0,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
100025,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典使馆举办中瑞青少年羽毛球交流开放日活动(2019-01-28),2019-01-28
100148,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典大使桂从友在2019年瑞典中国学生学者春节联欢会上的致辞(2019-01-21),2019-01-21
100222,乌兹别克斯坦,zh,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_乌兹别克斯坦_外交掠影_,费尔干纳——名符其实的乌兹别克斯坦明珠(2018-11-08),2018-11-08


In [76]:
dumptuple = [list(uploadDF.loc[x,]) for x in uploadDF[uploadDF['date']!='9999-99-99'].index]
dumptuple = [tuple(x) for x in dumptuple]

In [77]:
dumptuple[0]

('阿富汗',
 'zh',
 '相关新闻',
 'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/xgxw_676213/t1632522.shtml',
 '首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_',
 '外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25)',
 '2019-01-25')

In [78]:
len(dumptuple)

6432

In [79]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        port=5432,
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

sql = """
    INSERT INTO documents(
    doc_country,
    doc_language,
    doc_page_name,
    doc_url,
    doc_nav,
    doc_title,
    doc_date) VALUES(%s,%s,%s,%s,%s,%s,%s)
    """




cur.executemany(sql,dumptuple)

conn.commit()

cur.close()
conn.close()

In [95]:
uploadDF.head()

,country,lang,page,url,nav,title,date
0,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
100025,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典使馆举办中瑞青少年羽毛球交流开放日活动(2019-01-28),2019-01-28
100148,瑞典,zh,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_瑞典_驻外报道_,驻瑞典大使桂从友在2019年瑞典中国学生学者春节联欢会上的致辞(2019-01-21),2019-01-21
100222,乌兹别克斯坦,zh,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_乌兹别克斯坦_外交掠影_,费尔干纳——名符其实的乌兹别克斯坦明珠(2018-11-08),2018-11-08


In [96]:
nonDate.shape

(3, 6)

In [100]:
nonDate = uploadDF[uploadDF['date']=='9999-99-99'].copy()
nonDate = nonDate.drop(columns='date')
dumptuple = [list(nonDate.loc[x,]) for x in nonDate.index]
dumptuple = [tuple(x) for x in dumptuple]

In [101]:
dumptuple

[('法国',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679134/1206x2_679154/t1622990.shtml',
  '首页_国家和组织_国家（地区）_欧洲_法国_驻外报道_',
  '驻法国大使翟隽在第六届中法地方政府合作高层论坛开幕式上的致辞'),
 ('非洲联盟',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/lhg_683022/zwbd_683042/t1620005.shtml',
  '首页_国家和组织_国际和地区组织_非洲联盟_驻外报道_',
  '驻非盟使团团长刘豫锡大使会见非盟委员会主席办主任'),
 ('赤道几内亚',
  'zh',
  '相关新闻',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677510/xgxw_677516/t1168730.shtml',
  '首页_国家和组织_国家（地区）_非洲_赤道几内亚_相关新闻_',
  '赤道几内亚总统奥比昂会见外交部副部长张明')]

In [102]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        port=5432,
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

sql = """
    INSERT INTO documents(
    doc_country,
    doc_language,
    doc_page_name,
    doc_url,
    doc_nav,
    doc_title
    ) VALUES(%s,%s,%s,%s,%s,%s)
    """




cur.executemany(sql,dumptuple)

conn.commit()

cur.close()
conn.close()

In [107]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        port=5432,
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

sql = """
    SELECT doc_date
    FROM documents
    WHERE doc_html IS NULL AND doc_skip != 1
    LIMIT 20
    """




cur.execute(sql)
existing = cur.fetchall()
conn.commit()

cur.close()
conn.close()

In [108]:
existing

[(datetime.date(2009, 8, 10),),
 (datetime.date(2009, 8, 3),),
 (datetime.date(2009, 7, 24),),
 (datetime.date(2009, 7, 23),),
 (datetime.date(2012, 4, 3),),
 (datetime.date(2009, 7, 21),),
 (datetime.date(2009, 7, 6),),
 (datetime.date(2009, 7, 2),),
 (datetime.date(2009, 6, 16),),
 (datetime.date(2009, 6, 10),),
 (datetime.date(2009, 6, 10),),
 (datetime.date(2011, 11, 16),),
 (datetime.date(2009, 6, 4),),
 (datetime.date(2009, 6, 2),),
 (datetime.date(2009, 6, 1),),
 (datetime.date(2009, 6, 1),),
 (datetime.date(2009, 5, 22),),
 (datetime.date(2009, 5, 4),),
 (datetime.date(2009, 5, 4),),
 (datetime.date(2009, 5, 4),)]